Project Structure
- Setup Chunking and processing steps for inputs (PDFs, docx, txt)
- Setup PGVector Store (VS) + postgresql => Done
- Using LLMs (OpenAI, Gemini,...) to query vectors from VS

# Create PostgreDB 

In [1]:
import sys
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv('/Users/longnv/Coding/rag_llama_index/deployment/.env')

True

In [13]:
import psycopg2

db_name = os.environ['POSTGRES_DB']
host = "localhost"
password = os.environ['POSTGRES_PASSWORD']
port = "5432"
user = os.environ['POSTGRES_USER']
# conn = psycopg2.connect(connection_string)
conn = psycopg2.connect(
    dbname=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

# with conn.cursor() as c:
#     c.execute(f"DROP DATABASE IF EXISTS {db_name}")
#     c.execute(f"CREATE DATABASE {db_name}")

In [16]:
print(conn.status)
print(conn.get_dsn_parameters())
with conn.cursor() as cursor:
    cursor.execute("SELECT 1")
    print("Connection is active")

1
{'user': 'admin', 'channel_binding': 'prefer', 'dbname': 'rag_db', 'host': 'localhost', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslcertmode': 'allow', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'gssdelegation': '0', 'target_session_attrs': 'any', 'load_balance_hosts': 'disable'}
Connection is active


# Checking file input

In [ ]:
from pydantic import BaseModel, Field, field_validator
from pydantic_ai import Agent, RunContext
from uuid import UUID
from chonkie import TokenChunker, SemanticChunker

In [20]:
import os
from pathlib import Path
import PyPDF2
from chonkie import SemanticChunker
from sentence_transformers import SentenceTransformer


def extract_text_from_pdf(pdf_path):
    """
    Extract text from PDF file using PyPDF2.

    Args:
        pdf_path (str): Path to the PDF file

    Returns:
        str: Extracted text from all pages
    """
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text() + "\n"

    return text


def chunk_with_semantic_chunker(text, chunk_size=512, similarity_threshold=0.7, embedding_model=None):
    """
    Chunk text using Chonkie's SemanticChunker with custom embedding model.

    Args:
        text (str): Text to chunk
        chunk_size (int): Maximum tokens per chunk
        similarity_threshold (float): Similarity threshold for semantic chunking
        embedding_model: Custom embedding model (SentenceTransformer or similar)

    Returns:
        list: List of chunks
    """
    if embedding_model:
        chunker = SemanticChunker(
            chunk_size=chunk_size,
            similarity_threshold=similarity_threshold,
            embedding_model=embedding_model
        )
    else:
        # Use default embedding model
        chunker = SemanticChunker(
            chunk_size=chunk_size,
            similarity_threshold=similarity_threshold
        )

    chunks = chunker.chunk(text)
    return chunks


def save_chunks_to_file(chunks, output_path, chunker_type):
    """
    Save chunks to a text file.

    Args:
        chunks (list): List of chunks
        output_path (str): Output file path
        chunker_type (str): Type of chunker used
    """
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(f"Chunks created using {chunker_type}\n")
        f.write("=" * 50 + "\n\n")

        for i, chunk in enumerate(chunks, 1):
            f.write(f"Chunk {i}:\n")
            f.write("-" * 20 + "\n")
            f.write(f"{chunk.text}\n\n")
            f.write(f"Tokens: {chunk.token_count}\n")
            if hasattr(chunk, 'start_index'):
                f.write(f"Start Index: {chunk.start_index}\n")
            if hasattr(chunk, 'end_index'):
                f.write(f"End Index: {chunk.end_index}\n")
            f.write("\n" + "="*50 + "\n\n")

In [ ]:
print("\nStep 1: Loading documents")
texts = extract_text_from_pdf(r'D:\Project\rag_with_llama\docs\llama2.pdf')

# Create output directory
output_dir = "chunked_output"
os.makedirs(output_dir, exist_ok=True)

print("\nStep 2: Chunking with SemanticChunker")
semantic_chunks = chunk_with_semantic_chunker(
    texts,
    chunk_size=512,
    similarity_threshold=0.7
    )


Step 1: Loading documents

Step 2: Loading embedding models...


d:\Programs\Anaconda\envs\longnv\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Step 3: Chunking with SemanticChunker


In [ ]:
semantic_chunks[0]

Chunk(text='Llama 2 : Open Foundation and Fine-Tuned Chat Models
Hugo Touvron∗Louis Martin†Kevin Stone†
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
', token_count=45, start_index=0, end_index=167)

In [28]:
from typing import List, Dict, Any, Optional

class EmbeddingGenerator:
    """Generate embeddings using SentenceTransformers."""

    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        """
        Initialize embedding generator.

        Args:
            model_name: SentenceTransformer model name
        """
        self.model_name = model_name
        self.model = SentenceTransformer(model_name)
        self.embedding_dim = self.model.get_sentence_embedding_dimension()

    def embed_text(self, text: str) -> List[float]:
        """
        Generate embedding for a single text.
        Args:
            text: Input text
        Returns:
            List of embedding values
        """
        embedding = self.model.encode(text)
        return embedding.tolist()

    def embed_batch(self, texts: List[str]) -> List[List[float]]:
        """
        Generate embeddings for multiple texts.
        Args:
            texts: List of input texts
        Returns:
            List of embedding lists
        """
        embeddings = self.model.encode(texts)
        return [emb.tolist() for emb in embeddings]

embedding_model = 'all-MiniLM-L6-v2'
embedding_generator = EmbeddingGenerator(embedding_model)

d:\Programs\Anaconda\envs\longnv\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [29]:
embedding_generator.embed_text(semantic_chunks[0])

[[-0.029210299253463745,
  -0.008136039599776268,
  0.03420507535338402,
  0.04029562324285507,
  0.0742618590593338,
  0.05922506004571915,
  0.08180944621562958,
  0.03713741898536682,
  0.03291485831141472,
  -0.03143712878227234,
  0.060038577765226364,
  -0.07254353910684586,
  0.024705227464437485,
  -0.004340943414717913,
  -0.01321369782090187,
  0.018410881981253624,
  -0.0830569714307785,
  -0.014857964590191841,
  -0.12271387130022049,
  0.0023262666072696447,
  -0.03376968950033188,
  0.027208687737584114,
  -0.020789319649338722,
  0.02001935802400112,
  -0.01195456087589264,
  -0.01662687584757805,
  -0.021979205310344696,
  -0.004876245278865099,
  0.00128872727509588,
  -0.08734936267137527,
  0.007423588540405035,
  0.1130385547876358,
  0.0585951954126358,
  -0.019217325374484062,
  0.017139486968517303,
  -0.07475003600120544,
  -0.07950091361999512,
  -0.058088209480047226,
  0.03741077333688736,
  0.059456128627061844,
  -0.08294066041707993,
  -0.08918190002441406